#In this assignment, you'll continue working with the house prices data.

**1.** Load the houseprices data from Thinkful's database.

**2.** Reimplement your model from the previous checkpoint.

**3.** Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?

## **1. Load the housing prices data from Thinkful's database.**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sqlalchemy import create_engine
import seaborn as sns

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

house_prices_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

house_prices_df.head()


,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,...,centralair,electrical,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000


## **2. Reimplement your model from the previous checkpoint.**

In [3]:
house_prices_df1 = house_prices_df.loc[house_prices_df['saleprice'] < 350000]

# Y is the target variable
Y = house_prices_df1['saleprice']

# X is the feature set
X = house_prices_df1[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'yearremodadd', 'yearbuilt']]

# add a constant to the model
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

#print("The number of observations in training set is {}".format(X_train.shape[0]))
#print("The number of observations in test set is {}".format(X_test.shape[0]))

# add a constant to the model, the “baseline” of the data in case all of your features values are 0.

X_train = sm.add_constant(X_train)

# Let's train the model

results = sm.OLS(y_train, X_train).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     632.2
Date:                Tue, 21 Jul 2020   Prob (F-statistic):               0.00
Time:                        01:36:15   Log-Likelihood:                -13112.
No. Observations:                1123   AIC:                         2.624e+04
Df Residuals:                    1116   BIC:                         2.627e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.157e+06   1.03e+05    -11.223      0.000   -1.36e+06   -9.54e+05
overallqual   1.618e+04   1018.238     15.886      0.000    1.42e+04    1.82e+04
grlivarea       36.3774      2.259     16.106      0.000      31.946      40.809
garagecars    1.384e+04   1512.212      9.155      0.000    1.09e+04    1.68e+04
totalbsmtsf     17.5254      2.466      7.107      0.000      12.687      22.364
yearremodadd   316.5483     52.585      6.020      0.000     213.372     419.724
yearbuilt      257.2559     40.398      6.368      0.000     177.991     336.521
==============================================================================
Omnibus:                      583.428   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22861.124
Skew:                          -1.716   Prob(JB):                         0.00
Kurtosis:                      24.836   Cond. No.                     4.03e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.03e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## **3. Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?**

###**3.1. OLS**

In [4]:
# We fit an OLS model using sklearn
lrm = LinearRegression()
lrm.fit(X_train, y_train)


# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.7726742305375318
-----Test set statistics-----
R-squared of the model in the test set is: 0.7916420600437634
Mean absolute error of the prediction is: 20096.91847046615
Mean squared error of the prediction is: 775513252.369483
Root mean squared error of the prediction is: 27848.038573111087
Mean absolute percentage error of the prediction is: 12.48903979765544


Let's perform a log transformation on the dependent variable (saleprice)

In [5]:
# Y is the target variable
Y = np.log(house_prices_df1['saleprice'])

# X is the feature set
X = house_prices_df1[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'yearremodadd', 'yearbuilt']]

# add a constant to the model
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

# add a constant to the model, the “baseline” of the data in case all of your features values are 0.

X_train = sm.add_constant(X_train)

# Let's train the model

results = sm.OLS(y_train, X_train).fit()

#results.summary()



# We fit an OLS model using sklearn
lrm = LinearRegression()
lrm.fit(X_train, y_train)


# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

The number of observations in training set is 1123
The number of observations in test set is 281
R-squared of the model in the training set is: 0.7767246618024104
-----Test set statistics-----
R-squared of the model in the test set is: 0.801893701601791
Mean absolute error of the prediction is: 0.1140673129907533
Mean squared error of the prediction is: 0.0260109072541379
Root mean squared error of the prediction is: 0.1612789733788565
Mean absolute percentage error of the prediction is: 0.9579298561011598


In [6]:
from sklearn.model_selection import cross_val_score
ols_result = cross_val_score(lrm, X_train, y_train, cv=10)
print(ols_result)

print("Accuracy:",ols_result.mean()*100)

[0.84154629 0.83390049 0.82340118 0.47554602 0.76459214 0.69967769
 0.75771762 0.76485706 0.75386617 0.8494306 ]
Accuracy: 75.64535251203512


### **3.2 Ridge regression**

Selecting The Best Alpha Value In Ridge Regression

In [7]:
#Selecting The Best Alpha Value In Ridge Regression

from sklearn.linear_model import RidgeCV

# Create ridge regression with three possible alpha values
regr_cv = RidgeCV(alphas=[0.1, 1.0, 10.0])

# Fit the linear regression
model_cv = regr_cv.fit(X, Y)

# View alpha
model_cv.alpha_

0.1

In [8]:
from sklearn.linear_model import Ridge

# Fitting a ridge regression model. Alpha is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced.
ridgeregr = Ridge(alpha = model_cv.alpha_) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model on the training set is: 0.7767246599947509
-----Test set statistics-----
R-squared of the model on the test set is: 0.8018978442057729
Mean absolute error of the prediction is: 0.1140652991499132
Mean squared error of the prediction is: 0.026010363339639287
Root mean squared error of the prediction is: 0.16127728711644207
Mean absolute percentage error of the prediction is: 0.9579132318211231


In [9]:
regr_result = cross_val_score(ridgeregr, X_train, y_train, cv=10)
print(regr_result)

print("Accuracy:",regr_result.mean()*100)

[0.84155209 0.83389876 0.82341709 0.4755066  0.76459155 0.6996682
 0.75773123 0.76487466 0.75386258 0.84942945]
Accuracy: 75.64532214584486


### **3.3 Lasso**

In [10]:
#Selecting The Best Alpha Value In Ridge Regression

from sklearn.linear_model import LassoCV

# Create ridge regression with three possible alpha values
lasso_cv = LassoCV(alphas=[0.1, 1.0, 10.0])

# Fit the linear regression
model_cv = lasso_cv.fit(X, Y)

# View alpha
model_cv.alpha_

0.1

In [11]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=model_cv.alpha_) 
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model on the training set is: 0.7011984475001538
-----Test set statistics-----
R-squared of the model on the test set is: 0.7614062376009454
Mean absolute error of the prediction is: 0.12904881216828087
Mean squared error of the prediction is: 0.0313268193659497
Root mean squared error of the prediction is: 0.17699383990961295
Mean absolute percentage error of the prediction is: 1.0837660976369103


In [12]:
lassoregr_result = cross_val_score(lassoregr, X_train, y_train, cv=10)
print(lassoregr_result)

print("Accuracy:",lassoregr_result.mean()*100)

[0.81124397 0.72998743 0.814911   0.1653036  0.70754073 0.57999542
 0.7286915  0.75927826 0.67433712 0.77531493]
Accuracy: 67.46603974655208


### **3.4 ElasticNet regression**

In [13]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet(alpha=model_cv.alpha_, l1_ratio=0.5) 
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model on the training set is: 0.7316751042544716
-----Test set statistics-----
R-squared of the model on the test set is: 0.7819703034286623
Mean absolute error of the prediction is: 0.12247662908520171
Mean squared error of the prediction is: 0.028626804205717074
Root mean squared error of the prediction is: 0.1691945749890258
Mean absolute percentage error of the prediction is: 1.0290257187812024


In [14]:
elasticregr_result = cross_val_score(elasticregr, X_train, y_train, cv=10)
print(elasticregr_result)

print("Accuracy:",elasticregr_result.mean()*100)

[0.83009087 0.77002611 0.83844208 0.22065907 0.73015296 0.61611491
 0.75572864 0.78353941 0.70413833 0.80229811]
Accuracy: 70.51190499891374


###**Which model is the best? Why?**

In [15]:
fold = pd.DataFrame([ols_result*100, regr_result*100, lassoregr_result*100, elasticregr_result*100], index = ['OLS Model', 'Lasso Model', 'Ridge Model', 'ElasticNet Model'])

fold.columns = ['Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_8','Fold_10']

fold['avg_score'] = fold.mean(axis=1)

fold.sort_values(['avg_score'],ascending=False)

,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Fold_6,Fold_7,Fold_8,Fold_8,Fold_10,avg_score
OLS Model,84.154629,83.390049,82.340118,47.554602,76.459214,69.967769,75.771762,76.485706,75.386617,84.943060,75.645353
Lasso Model,84.155209,83.389876,82.341709,47.550660,76.459155,69.966820,75.773123,76.487466,75.386258,84.942945,75.645322
ElasticNet Model,83.009087,77.002611,83.844208,22.065907,73.015296,61.611491,75.572864,78.353941,70.413833,80.229811,70.511905
Ridge Model,81.124397,72.998743,81.491100,16.530360,70.754073,57.999542,72.869150,75.927826,67.433712,77.531493,67.466040


**OLS**

```
R-squared of the model in the training set is: 0.7767246618024104
-----Test set statistics-----
R-squared of the model in the test set is: 0.801893701601791
Mean absolute error of the prediction is: 0.1140673129907533
Mean squared error of the prediction is: 0.0260109072541379
Root mean squared error of the prediction is: 0.1612789733788565
Mean absolute percentage error of the prediction is: 0.9579298561011598
```
 **Ridge**  
```
R-squared of the model on the training set is: 0.7767246599947509
-----Test set statistics-----
R-squared of the model on the test set is: 0.8018978442057729
Mean absolute error of the prediction is: 0.1140652991499132
Mean squared error of the prediction is: 0.026010363339639287
Root mean squared error of the prediction is: 0.16127728711644207
Mean absolute percentage error of the prediction is: 0.9579132318211231
```
**Lasso** 
```
R-squared of the model on the training set is: 0.7011984475001538
-----Test set statistics-----
R-squared of the model on the test set is: 0.7614062376009454
Mean absolute error of the prediction is: 0.12904881216828087
Mean squared error of the prediction is: 0.0313268193659497
Root mean squared error of the prediction is: 0.17699383990961295
Mean absolute percentage error of the prediction is: 1.0837660976369103
```
 **ElasticNet**
```
R-squared of the model on the training set is: 0.7316751042544716
-----Test set statistics-----
R-squared of the model on the test set is: 0.7819703034286623
Mean absolute error of the prediction is: 0.12247662908520171
Mean squared error of the prediction is: 0.028626804205717074
Root mean squared error of the prediction is: 0.1691945749890258
Mean absolute percentage error of the prediction is: 1.0290257187812024
```

The cross-validation scores made it easy to pick out the best model.
Linear regression and Ridge are identical and comes out on top